<a href="https://colab.research.google.com/github/Antoine-Carre/-movie_imdb_project_2-/blob/main/BDD_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive")
import pandas as pd
import re
import numpy as np
from datetime import datetime, timedelta
import time
import plotly.express as px

Mounted at /content/drive


This document is usefull for BDD python : https://nonoo22222.herokuapp.com/

All data from this file have to be download in the same folder (created a Folder on google Drive with the datetoday writing like this: 

In [ ]:
date_today = str(datetime.now().year) + str(datetime.now().month)+ str(datetime.now().day) 
date_today

'202233'

In [ ]:
test = "20220301"
# if there isn't new folder -> using test and not 
path = "/content/drive/Shareddrives/Solinum/Mesure d'impact/analyse code/metabase/{}".format(20220301)

In [ ]:
# Search csv file from the search done before 2021
sAVANT = pd.read_csv("/content/drive/Shareddrives/Solinum/Mesure d'impact/analyse code/metabase/20211030/searchAVANTcopy2.csv")

# Importation files


In [ ]:
# Les fichiers sont récupérés au format json depuis la mongoDB avec la ligne de code suivante:
# mongoexport --db <database-name> --collection <collection-name> --out output.json

# Soliguide users (there is Soliguide and pro)
users = pd.read_json("/content/drive/Shareddrives/Solinum/Mesure d'impact/analyse code/metabase/20220301/users.json", lines=True) #lines=True

# Mails send to users for summer update
#mailsJSON_ok = pd.read_json("/content/drive/Shareddrives/Solinum/Mesure d'impact/analyse code/metabase/20220301/mails.json", lines=True) #lines=True

# All update in Soliguide
historyJSON = pd.read_json("/content/drive/Shareddrives/Solinum/Mesure d'impact/analyse code/metabase/20220301/history.json", lines=True) #lines=True

# All lieux in Soliguide (fiches = structure)
fichesJSON = pd.read_json("/content/drive/Shareddrives/Solinum/Mesure d'impact/analyse code/metabase/20220301/fichesJSON.json", lines=True) #lines=True

# All organization

orgaJSON = pd.read_json("/content/drive/Shareddrives/Solinum/Mesure d'impact/analyse code/metabase/20220301/orga.json", lines=True) #lines=True
#orgaJSON["Created At"] = orgaJSON["Created At"].apply(lambda x: datetime.strptime(x.split(".")[0].split("Z")[0], "%Y-%m-%dT%H:%M:%S"))
#orga = orgaJSON

# Ce fichier provient de la base "Search" de metabase

s = pd.read_json("/content/drive/Shareddrives/Solinum/Mesure d'impact/analyse code/metabase/20220301/search.json", lines=True)

In [ ]:
orgaJSON['Created At'] = orgaJSON['createdAt'].apply(pd.Series)

In [ ]:
orgaJSON["created At"] = orgaJSON["Created At"].astype(str).apply(lambda x: datetime.strptime(x.split(".")[0].split("Z")[0], "%Y-%m-%dT%H:%M:%S"))
orga = orgaJSON

# I - Creating Dataset for figure 1

### A - Les catégories spécifiques sur Soliguide: toilettes (304), wifi (502), fontaine (605), prise (503)

In [ ]:
# This function allow us to have a specific columns with list of categories per structure

def categorieNb(x):
  L = []
  if type(x) != list:
    return(np.nan)
  else:
    for services in x:
      L.append(services['categorie'])
  return(L)

fichesJSON["categorieNb"] = fichesJSON["services_all"].apply(lambda x: np.unique(categorieNb(x)))

In [ ]:
# dictionnary with bigCategorie

bigCatDictNb = {"Santé" : ['100', '101', '102', '103', '104', '105','106', '107', '108', '109', '110', '111', '1100','1101','1102', '1103','1104','1105','1106', '1107','1108','1109','1110','1111', '1112', '1113','1114','1115','1116','1117','1118','1119','1120','1121','1122'],
"Formation" : ['200', '201', '202', '203', '204', '205', '206'],
"Hygiène" : ['300', '301', '302', '303', '304', '305', '306'],
"Conseil" : ['400', '401', '402', '403', '404', '405', '406', "407"],
"Technologie" : ['500', '501', '502', '503', '504', '505', '506'],
"Alimentation" : ['600', '601', '602', '603', '604', '605', '606'],
"Accueil" : ['700', '701', '702', '703', '704', '705', '706', '707', '708'],
"Activités": ['800', '801', '802', '803', '804', '805'],
"Matériel" : ['900', '901', '902', '903', '904']}

# This function able to have a specific columns with Big categories

def bigCatDictNb_func(x):
  L = []
  for cat in x:
    for keys in bigCatDictNb.keys():
      if str(cat) in bigCatDictNb[keys]:
        L.append(keys)
  return(L)

fichesJSON["bigCatégorie"] = fichesJSON["categorieNb"].apply(lambda x: bigCatDictNb_func(x))

In [ ]:
# This function make a specific treatment for categories which doesn't ndf_fiches_3 =pd.json_normalize(df_fiches['services_all'])
#eed to be updated

def  nonmaj(x):  
  '''Input: list of categories
  Output: True if the categorie isn't necessary to be update (toilet fountain wifi) else False'''

  if x == [304]:
    return(True)
  elif x == [502]:
    return(True)
  elif x == [605]:
    return(True)
  elif x == [503]:
    return(True)
  elif (x == [304,502]) or (x == [304,502]):
    return(True)
  elif (x == [304,605]) or (x == [304,605]):
    return(True)
  elif (x == [502,605]) or (x == [502,605]):
    return(True)
  elif (x == [503,304]) or (x == [503,502]) or (x == [503,605]):
    return(True)
  elif (x == [304, 503]) or (x == [502, 503]) or (x == [605, 503]):
    return(True)
  elif (x == [502,605, 304]) or (x == [502,304, 605]) or (x == [304,502,605]) or (x == [304, 605, 502]) or (x == [605, 304,502]) or (x == [605, 502, 304]):
    return(True)
  else:
    return(False)


# Creation of a new columns categorieNonMaj if the categorie isn't necessary to be update (toilet fountain wifi) 
fichesJSON["categorieNonMaj"] = fichesJSON["categorieNb"].apply(lambda x: nonmaj(list(x)))


# This columns give territory where there is Soliguide implanted 
TerritoireIn = ["Paris", "Seine-Saint-Denis", "Gironde", "Seine-et-Marne", "Loire-Atlantique", "Hauts-de- Seine", "Essonne", 'Hauts-de-Seine', "Yvelines", 'Val-de-Marne', "Bas-Rhin", "Val d’Oise", "Val-d'Oise", 'Département de Paris', 'Alpes-Maritimes', 'Arrondissement de Paris', 'Ain',"Nord" ,"Morbihan", "Côte-d'Or", "Alpes-Maritimes", "Bouches-du-Rhône", "Ardèche", "Cantal", "Puy-de-Dôme", "Hérault", "Charente", "Haute-Vienne", "Dordogne", "Indre", "Seine-Maritime"]
fichesJSON["DepartementIn"] = fichesJSON["departement"].apply(lambda x: True if x in TerritoireIn else False)

### B - Cleaning data from fiches and crossing data from orga

In [ ]:
# Cleaning the column orga.places to get rid of dict with ObjectId in list and only get all the _id
temp_df = orga["places"].apply(pd.Series)

for n in range(len(temp_df.columns)):
      temp_df[n] = temp_df[n].apply(pd.Series).drop(columns=0)

In [ ]:
# Merging all columns with place_id into one column with place_ids
temp_df
temp_df['places'] = temp_df[temp_df.columns[0:]].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1
)
orga["places"] = temp_df.places
orga["places"] = orga["places"].to_list()

In [ ]:
orga["places"] =  [x.split(',') for x in orga["places"]]

In [ ]:
orga["_id"] = pd.json_normalize(orga["_id"])

In [ ]:
fichesJSON["_id"] = pd.json_normalize(fichesJSON["_id"])

In [ ]:
# Creation of a columns with the list of id Orga from the id of Fiches

# This line just mentionned that places contains idFiche
orga["idFiche"] = orga["places"]

def idOrga_idFiche_fct(orga = orga):
  ''' Keys: idFiche
  Value: List of id Orga ''' 

  # Initialisation
  idOrga_idFiche = {}
  for index in orga.index:
    for idFiche in orga["idFiche"][index]:
      idOrga_idFiche[idFiche] = []

  # Creation of the dict
  for index in orga.index:
    for idFiche in orga["idFiche"][index]:
      idOrga_idFiche[idFiche].append(orga["_id"][index])
  return(idOrga_idFiche)

dictTous = idOrga_idFiche_fct()

fichesJSON["idOrgaTous"] = fichesJSON["_id"].apply(lambda x: dictTous[x] if x in dictTous.keys() else np.nan)


In [ ]:
def compteProId(x):
  '''Input: ID of fiches
  Output: return "Actif" if the record is at least linked to an active account 
  return "Non actif" if the record is linked to an account but this is not an active account
  return "Sans orga" if the list of Organization is empty'''


  if type(x) == list:
    for i in x: 
      if i in list(orgaActive["_id"]):
        return("Actif")
    return("Non actif")
  else:
    return("Sans orga")

In [ ]:
def compteProId_nb(x):
  '''Input: ID of fiches
  Output: return "Actif" if the record is at least linked to an active account 
  return "Non actif" if the record is linked to an account but this is not an active account
  return "Sans orga" if the list of Organization is empty'''
  L = 0
  if type(x) == list:
    for i in x: 
      if i in list(orgaActive["_id"]):
        L += 1
  return(L)


In [ ]:
def compteProDate(x):
  '''Input: ID of fiches
  Output: return "Actif" if the record is at least linked to an active account 
  return "Non actif" if the record is linked to an account but this is not an active account
  return "Sans orga" if the list of Organization is empty'''


  if type(x) == list:
    for i in x: 
      if i in list(orgaActive["_id"]):
        return("Actif")
    return("Non actif")
  else:
    return("Sans orga")

In [ ]:
# Cleaning the column orga.users to get rid of dict with ObjectId in list and only get all the _id
temp_df = orga["users"].apply(pd.Series)

for n in range(len(temp_df.columns)):
      temp_df[n] = temp_df[n].apply(pd.Series).drop(columns=0)

temp_df['users'] = temp_df[temp_df.columns[0:]].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1
)
orga["users"] = temp_df.users
orga["users"] = orga["users"].to_list()

orga["users"] =  [x.split(',') for x in orga["users"]]

In [ ]:
users.status.value_counts()

PRO                8835
SIMPLE_USER         134
ADMIN_SOLIGUIDE      63
API_USER              8
ADMIN_TERRITORY       3
Name: status, dtype: int64

In [ ]:
users["_id"] = pd.json_normalize(users["_id"])

In [ ]:
# This is the list of users in Soliguide 
usersIDList = list(users["_id"])

def comptePro(OwnerList):
  ''' This function is usefull for the columns owner of organization
  Input: The list of owner
  Output: True if there is at least one owner in the users. '''

  L = []
  for id in OwnerList:
    if id in usersIDList:
      L.append(True)
    else:
      L.append(False)
  # It may be interesting to return L to see the proportion of [True, False] for multiples owners.
  if True in L:
    return(True)
  else:
    return(False)

# I created a new columns ComptePro which is an application of comptePro function 
orga["comptePro"] = orga["users"].apply(lambda x: comptePro(x))

# I selected the dataFrame where compte pro is True
orgaActive = orga[orga["comptePro"] == True]


In [ ]:
# Creation of the new columns compteType which use compteProId function
fichesJSON["compteType"] = fichesJSON["idOrgaTous"].apply(lambda x: compteProId(x))
fichesJSON["nbCompteType"] = fichesJSON["idOrgaTous"].apply(lambda x: compteProId_nb(x))

In [ ]:
fichesJSON["compteType"].value_counts()

Sans orga    6126
Actif        6006
Non actif    4838
Name: compteType, dtype: int64

In [ ]:
#ES = { FERME: 1, FERME_TEMP: 2, OUVERT: 0 }
def closeType_value(x):
  if x == 0:
    return("OPEN")
  elif x == 2:
    return("CLOSE_TEMP")
  elif x == 1:
    return("CLOSE")
  else:
    return("NO_INFO")

In [ ]:
fichesJSON["CloseType"] = fichesJSON["close"].apply(lambda x: closeType_value(x["closeType"]))


In [ ]:
fichesJSON["createdAt"] = pd.json_normalize(fichesJSON["createdAt"])

In [ ]:
fichesJSON["Created At"] = fichesJSON["createdAt"].apply(lambda x: datetime.strptime(x.split(".")[0].split("Z")[0], "%Y-%m-%dT%H:%M:%S"))


In [ ]:
fiches = fichesJSON

In [ ]:
def general(df, columns= "year-month"):
  a = pd.DataFrame(df[columns].value_counts())
  a["date"] = a.index
  return(a.sort_values(by="date"))

In [ ]:
fiches['Code Postal str'] = fiches['codePostal'].apply(lambda x: str(x))

In [ ]:
fiches["updatedAt ok"] = pd.json_normalize(fichesJSON["updatedAt"])

In [ ]:
import datetime
fiches["createdAt ok"] = fiches["Created At"].astype(str).apply(lambda x: x.split("T")[0])
fiches["createdAt ok"] = fiches["Created At"].astype(str).apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
fiches["year-month"] = fiches["Created At"].apply(lambda x: datetime.datetime(x.year, x.month, 1))
fiches["updatedAt ok"] = pd.json_normalize(fichesJSON["updatedAt"])
fiches["updatedAt ok"] = fiches["updatedAt ok"].astype(str).apply(lambda x: x.split("T")[0])
fiches["updatedAt ok"] = fiches["updatedAt ok"].astype(str).apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
fiches["year-month UPDATE"] = fiches["updatedAt ok"].apply(lambda x: datetime.datetime(x.year, x.month, 1))
fiches["Territoire"] = fiches['Code Postal str'].apply(lambda x: x[:2] if type(x)!= float else x)

In [ ]:
def renameDf(colonne, df):
  dictRename = {}
  for dep in list(df.columns) :
    dictRename[dep] = '{} (dep:{})'.format(colonne, dep)
  return(df.rename(columns=dictRename))

In [ ]:
def cleaning(x):
  if type(x) == str:
    return(x.replace(" ", "").replace("-", "").replace("'", "").replace("/", "").replace("’", "").replace("à", "a").replace("é", "e").replace("è", "e").replace("ê", "e").lower().split("(")[0])
  else: 
    return(x)

In [ ]:
df = pd.DataFrame(fiches[(fiches['CloseType'] != "CLOSE") & (fiches["categorieNonMaj"] == False) & (fiches["statut"] == 1)]["year-month UPDATE"].value_counts())
df["date"] = df.index
df = df.sort_values(by="date", ascending=False)
df["cum"] = df["year-month UPDATE"].cumsum()

In [ ]:
crossCompte = pd.crosstab(fiches[(fiches['CloseType'] != "CLOSE") & (fiches["categorieNonMaj"] == False) & (fiches["compteType"] == "Actif")]["year-month"], fiches[(fiches['CloseType'] == '0') & (fiches["categorieNonMaj"] == False) & (fiches["compteType"] == "Actif")]["Territoire"])
crossCompte = renameDf('Fiches reliées aux comptes pros', crossCompte)

crossSuivies = pd.crosstab(fiches[(fiches['CloseType'] != "CLOSE") & (fiches["categorieNonMaj"] == False)]["year-month"], fiches[(fiches['CloseType'] != "CLOSE") & (fiches["categorieNonMaj"] == False)]["Territoire"])
crossSuivies = renameDf('Suivies', crossSuivies)

crossEnligne = pd.crosstab(fiches[(fiches['CloseType'] != "CLOSE") & (fiches["categorieNonMaj"] == False) & (fiches["statut"] == 1)]["year-month"], fiches[(fiches['CloseType'] != "CLOS.E") & (fiches["categorieNonMaj"] == False) & (fiches["statut"] == 1)]["Territoire"])
crossEnligne = renameDf('En ligne', crossEnligne)

crossBrouillon = pd.crosstab(fiches[(fiches['CloseType'] != "CLOSE") & (fiches["categorieNonMaj"] == False) & (fiches["statut"] == 0)]["year-month"], fiches[(fiches['CloseType'] != "CLOSE") & (fiches["categorieNonMaj"] == False) & (fiches["statut"] == 0)]["Territoire"])
crossBrouillon = renameDf('En brouillon', crossBrouillon)

crossPro = pd.crosstab(fiches[(fiches['CloseType'] != "CLOSE") & (fiches["categorieNonMaj"] == False) & (fiches["statut"] == 2)]["year-month"], fiches[(fiches['CloseType'] != "CLOSE") & (fiches["categorieNonMaj"] == False) & (fiches["statut"] == 2)]["Territoire"])
crossPro = renameDf('Reservées aux pros', crossPro)

crossUpdate = pd.crosstab(fiches[(fiches['CloseType'] != "CLOSE") & (fiches["categorieNonMaj"] == False)]["year-month UPDATE"], fiches[(fiches['CloseType'] != "CLOSE") & (fiches["categorieNonMaj"] == False)]["Territoire"])
crossUpdate = renameDf('Mise à jour', crossUpdate)

crossCompte = pd.crosstab(fiches[(fiches['CloseType'] != "CLOSE") & (fiches["categorieNonMaj"] == False) & (fiches["compteType"] == "Actif")]["year-month"], fiches[(fiches['CloseType'] != "CLOSE") & (fiches["categorieNonMaj"] == False) & (fiches["compteType"] == "Actif")]["Territoire"])
crossCompte = renameDf('Fiches reliées aux comptes pros', crossCompte)


generalSuivies = general(fiches[(fiches['CloseType'] != "CLOSE") & (fiches["categorieNonMaj"] == False)]).rename(columns = {"year-month": "Suivies général"})
generalEnligne = general(fiches[(fiches['CloseType'] != "CLOSE") & (fiches["categorieNonMaj"] == False) & (fiches["statut"] == 1)]).rename(columns = {"year-month": "En ligne général"})
generalBrouillon = general(fiches[(fiches['CloseType'] != "CLOSE") & (fiches["categorieNonMaj"] == False) & (fiches["statut"] == 0)]).rename(columns = {"year-month": "Brouillon général"})
generalPro = general(fiches[(fiches['CloseType'] != "CLOSE") & (fiches["categorieNonMaj"] == False) & (fiches["statut"] == 2)]).rename(columns = {"year-month": "Reservées aux pros général"})
generalUpdate = general(fiches[(fiches['CloseType'] != "CLOSE") & (fiches["categorieNonMaj"] == False)], "year-month UPDATE").rename(columns = {"year-month UPDATE": "Mise à jour général"})
generalCompte = general(fiches[(fiches['CloseType'] != "CLOSE") & (fiches["categorieNonMaj"] == False) & (fiches["compteType"] == "Actif")]).rename(columns = {"year-month": "Fiches reliées aux comptes pros général"})

generalAll = generalSuivies.merge(right = generalEnligne, on = "date", how = 'outer').merge(right = generalBrouillon, on = "date", how = 'outer').merge(right = generalPro, on = "date", how = 'outer').merge(right = generalUpdate, on = "date", how = 'outer').merge(right = generalCompte, on = "date", how = 'outer').fillna(0)
generalAll.index = generalAll["date"]
generalAll = generalAll.drop(columns="date")

allFiches = pd.concat([generalAll, crossSuivies, crossEnligne, crossBrouillon, crossPro, crossUpdate, crossCompte], axis=1).fillna(0)
allFiches["year-month"] = allFiches.index

#with open("/content/drive/Shareddrives/Solinum/Mesure d'impact/analyse code/metabase/CrossTab.csv", 'w') as f: pd.concat([crossSuivies, crossEnligne, crossBrouillon, crossPro, crossUpdate, crossCompte], axis=1).fillna(0).to_csv(f)




In [ ]:
from datetime import datetime
from datetime import date

def datePresentation(allFiches, colEnter = "year-month", colExit = "datePresentation"):
  "This code available to transform simple date in end and start date for more comprehension"

  year2022 = ['2022-01-01','2022-02-01','2022-03-01','2022-04-01','2022-05-01','2022-06-01','2022-07-01','2022-08-01','2022-09-01','2022-10-01','2022-11-01','2022-12-01']
  year2021 = ['2021-01-01','2021-02-01','2021-03-01','2021-04-01','2021-05-01','2021-06-01','2021-07-01','2021-08-01','2021-09-01','2021-10-01','2021-11-01','2021-12-01']
  year2020 = ['2020-01-01','2020-02-01','2020-03-01','2020-04-01','2020-05-01','2020-06-01','2020-07-01','2020-08-01','2020-09-01','2020-10-01','2020-11-01','2020-12-01']
  year2019 = ['2019-01-01','2019-02-01','2019-03-01','2019-04-01','2019-05-01','2019-06-01','2019-07-01','2019-08-01','2019-09-01','2019-10-01','2019-11-01','2019-12-01']
  year2018 = ['2018-01-01','2018-02-01','2018-03-01','2018-04-01','2018-05-01','2018-06-01','2018-07-01','2018-08-01','2018-09-01','2018-10-01','2018-11-01','2018-12-01']
  chosenDate = year2018 + year2019 + year2020 + year2021 + year2022

  todé = date.today()
  if todé.month < 10:
    dateToday = (str(todé.year)+ "-0"+ str(todé.month)+"-"+"01")
  else:
    dateToday = (str(todé.year)+ "-"+ str(todé.month)+"-"+"01")
  if dateToday not in chosenDate:
    print("error you have to update choseDate")

  datePresentation = []
  for index in allFiches[colEnter].index:
    if allFiches[colEnter][index] != "no":
      for i in range(len(chosenDate)):
        if chosenDate[i] == str(allFiches[colEnter][index]).split(" ")[0]:
          if chosenDate[i] == dateToday:
            datePresentation.append("du " +  chosenDate[i] + " au " + str(todé))
          else:
            datePresentation.append("du " +  chosenDate[i] + " au " + chosenDate[i+1])

    else:
      datePresentation.append(np.nan)


  print("(len(datePresentation)", len(datePresentation),"allFiches.shape[0]",allFiches.shape[0])
  if len(datePresentation) != allFiches.shape[0]:
    print("error no same length between datePresentation and year-month (colEnter) ")
    return("error")
  else:
    allFiches[colExit] = datePresentation
    return(allFiches)


colCum = ["datePresentation"]
for col in allFiches.columns:
   allFiches["{} cumulé".format(col)] = allFiches["{}".format(col)].cumsum(axis=0) 
   colCum.append("{} cumulé".format(col))

# allFiches

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


###  E - Registration in the file of the *month*

In [ ]:
TerritoireSoliguide = ['datePresentation', 'général', '06','07','13','15', '16', '21', '24', '33','34','35','36', '44','56','59','63', '67', '75','76','77', '78', '87', '91', '92', '93', '94', '95']
L = []
for col in allFiches.columns:
  for Territoire in TerritoireSoliguide:
    if Territoire in col:
      L.append(col)

donnee_restreintes = datePresentation(allFiches, colEnter = "year-month", colExit = "datePresentation")[L]

with open("{}/fiche_figure1.csv".format(path), 'w') as f: donnee_restreintes.to_csv(f)


(len(datePresentation) 51 allFiches.shape[0] 51


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


# II - Display (figure 3)

### B - Cleaning data from Orga

Verified columns is kind of the same than comptePro function, it give the date of creation of the user, so it gives the date of creation of the compte pro

In [ ]:
# Changing format of column users["createdAt"] to datetime
users["createdAt"] = pd.json_normalize(users["createdAt"])
users["createdAt"] = pd.to_datetime(users["createdAt"])

In [ ]:
# Here we are keeping information from users in organization to define compte pro

ID_name = {}
ID_creation = {}
ID_SOLIGUIDE = {}
ID_creation_PRO = {}
for index in users.index:
  ID_name[users["_id"][index]] = users["name"][index]
  ID_creation[users["_id"][index]] = users["createdAt"][index]
  ID_SOLIGUIDE[users["_id"][index]] = users["status"][index]
  if users["status"][index] != "ADMIN_SOLIGUIDE":
    ID_creation_PRO[users["_id"][index]] = users["createdAt"][index]
  else:
    ID_creation_PRO[users["_id"][index]] = np.nan


def owner_name(x, dictName=ID_name):
  L = []
  for id in x:
    if id in dictName:
      L.append(dictName[id])
    else:
      pass
  return(L)


orga["Owner_creation"] = orga["users"].apply(lambda x: owner_name(x, ID_creation))
orga["Owner_creation_pro"] = orga["users"].apply(lambda x: owner_name(x, ID_creation_PRO))
orga["Owner_SOLIGUIDE"] = orga["users"].apply(lambda x: owner_name(x, ID_SOLIGUIDE) )

def owner_creation_time_pro_function(x):
  if len(x) == 0:
    return(False)
  else:
    for elem in x:
      if type(elem) != float:
        return(elem)
    return(False)

orga["Owner_creation_time_pro"] = orga["Owner_creation_pro"].apply(lambda x: owner_creation_time_pro_function(x))

# this is a really important columns, it gives the date of creation of the oldest owner in the organization
orga["Owner_creation_time"] = orga["Owner_creation"].apply(lambda x: x[0] if len(x) > 0 else np.nan)

### C -  Cleaning verified

In [ ]:
# orga["Owner_creation_time_pro_no_day"] = orga["Owner_creation_time_pro"].apply(lambda x: str(x).split("-")[0]+"-"+str(x).split("-")[1] if type(x) != bool else x)

In [ ]:
# orga.info()

### D -  Small display detail for figure 3

In [ ]:
# Extraire les compte pro de la table users
Cpte_pro = users[['createdAt','status','territories', 'typeAccount']]
Cpte_pro_pro = Cpte_pro[Cpte_pro.status == 'PRO']
Cpte_pro_pro = Cpte_pro_pro[Cpte_pro_pro.typeAccount == 'USER']


# Modifier le date pour ne garder que le mois et l'année
Cpte_pro_pro['createdAt'] = Cpte_pro_pro['createdAt'].dt.strftime('%Y-%m')

In [ ]:
Cpte_pro_pro.reset_index(inplace=True)

In [ ]:
# On retire les crochets de la colonne territoire
Cpte_pro_pro['territories'] = Cpte_pro_pro.territories.astype(str).str[2:-2]

In [ ]:
Cpte_pro_pro['territories'] = Cpte_pro_pro['territories'].map(lambda x: x.lstrip("00', '"))
Cpte_pro_pro['territories'] = Cpte_pro_pro['territories'].str.replace("', '", ",")
Cpte_pro_pro['territories'] = Cpte_pro_pro['territories'].str.replace(",", "")
Cpte_pro_pro['territories'] = Cpte_pro_pro['territories'].astype(str).str[:2]
Cpte_pro_pro['territories'].replace({'6':'06'}, inplace=True)

In [ ]:
Cpte_pro_pro.territories.value_counts()

75    403
06    244
92    218
94    216
33    214
78    213
67    176
44    175
93    161
77    156
34    116
95     94
36     61
13     58
91     54
7      34
63     32
76     22
       19
15     17
59     13
16     13
21      3
26      3
56      3
87      2
24      2
83      1
2A      1
69      1
Name: territories, dtype: int64

In [ ]:
comptePro = Cpte_pro_pro.pivot_table(index=['createdAt'], columns='territories', aggfunc='count')
comptePro = comptePro.status.reset_index()

In [ ]:
comptePro

territories,createdAt,,06,13,15,16,21,24,26,2A,33,34,36,44,56,59,63,67,69,7,75,76,77,78,83,87,91,92,93,94,95
0,2020-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.0,NaN,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-12,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71.0,NaN,NaN,41.0,NaN,NaN,NaN,31.0,1.0,NaN,131.0,NaN,NaN,52.0,NaN,NaN,NaN,39.0,55.0,49.0,NaN
2,2021-01,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,3.0,NaN,1.0,NaN,10.0,NaN,NaN,17.0,NaN,NaN,4.0,NaN,NaN,NaN,12.0,3.0,2.0,1.0
3,2021-02,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,3.0,NaN,NaN,NaN,11.0,NaN,NaN,15.0,NaN,NaN,6.0,NaN,NaN,NaN,21.0,6.0,5.0,8.0
4,2021-03,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,13.0,NaN,NaN,NaN,6.0,NaN,NaN,38.0,NaN,NaN,24.0,NaN,NaN,NaN,27.0,24.0,23.0,6.0
5,2021-04,1.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,NaN,2.0,16.0,NaN,NaN,NaN,15.0,NaN,NaN,5.0,NaN,2.0,9.0,NaN,NaN,NaN,8.0,12.0,8.0,6.0
6,2021-05,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,NaN,1.0,NaN,NaN,NaN,6.0,NaN,NaN,6.0,NaN,2.0,1.0,NaN,NaN,2.0,2.0,6.0,7.0,3.0
7,2021-06,NaN,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.0,NaN,NaN,24.0,NaN,3.0,NaN,15.0,NaN,NaN,44.0,NaN,25.0,24.0,NaN,NaN,6.0,28.0,22.0,39.0,12.0
8,2021-07,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,NaN,NaN,8.0,NaN,1.0,NaN,1.0,NaN,NaN,21.0,NaN,16.0,5.0,NaN,NaN,6.0,17.0,6.0,19.0,12.0
9,2021-08,NaN,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,17.0,4.0,NaN,NaN,2.0,NaN,3.0,3.0,3.0


In [ ]:
datePres = {"2022-03-01": "du 2022-02-01 au 2022-04-01",
            "2022-02-01": "du 2022-02-01 au 2022-03-01",
            "2022-01-01": "du 2022-01-01 au 2022-02-01",
            "2021-12-01": "du 2021-12-01 au 2022-01-01",
            "2021-11-01": "du 2021-11-01 au 2021-12-01",
            "2021-10-01": "du 2021-10-01 au 2021-11-01",
            "2021-09-01": "du 2021-09-01 au 2021-10-01",
            "2021-08-01": "du 2021-08-01 au 2021-09-01",
            "2021-07-01": "du 2021-07-01 au 2021-08-01",
            "2021-06-01": "du 2021-06-01 au 2021-07-01",
            "2021-05-01": "du 2021-05-01 au 2021-06-01",
            "2021-04-01": "du 2021-04-01 au 2021-05-01",
            "2021-03-01": "du 2021-03-01 au 2021-04-01",
            "2021-02-01": "du 2021-02-01 au 2021-03-01",
            "2021-01-01": "du 2021-01-01 au 2021-02-01",
            "2020-12-01": "du 2021-12-01 au 2021-01-01",
            "2020-11-01": "du 2021-11-01 au 2021-12-01"}
datePres2 = {"2022-03" : "mars 2022",
            "2022-02" : "février 2022",
            "2022-01" : "janvier 2022",
            "2021-12" : "décembre 2021",
            "2021-11" : "novembre 2021",
            "2021-10" : "octobre 2021",
            "2021-09": "septembre 2021", 
            "2021-08": "août 2021",
            "2021-07": "juillet 2021",
            "2021-06": "juin 2021",
            "2021-05": "mai 2021",
            "2021-04": "avril 2021",
            "2021-03": "mars 2021",
            "2021-02": "février 2021",
            "2021-01": "janvier 2021",
            "2020-12": "décembre 2020",
            "2020-11": "novembre 2020"}
comptePro['index1'] = comptePro.index

comptePro["datePresentation"] = comptePro.createdAt.apply(lambda x: datePres2[x])

In [ ]:
colCumCompte = ["datePresentation"]
for col in comptePro.columns:
   comptePro["{} cumulé".format(col)] = comptePro[col].cumsum(axis=0) 
   colCumCompte.append("{} cumulé".format(col))

In [ ]:
comptePro

territories,createdAt,,06,13,15,16,21,24,26,2A,33,34,36,44,56,59,63,67,69,7,75,76,77,78,83,87,91,92,93,94,95,index1,datePresentation,createdAt cumulé,cumulé,06 cumulé,13 cumulé,15 cumulé,16 cumulé,21 cumulé,24 cumulé,26 cumulé,2A cumulé,33 cumulé,34 cumulé,36 cumulé,44 cumulé,56 cumulé,59 cumulé,63 cumulé,67 cumulé,69 cumulé,7 cumulé,75 cumulé,76 cumulé,77 cumulé,78 cumulé,83 cumulé,87 cumulé,91 cumulé,92 cumulé,93 cumulé,94 cumulé,95 cumulé,index1 cumulé,datePresentation cumulé
0,2020-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.0,NaN,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,novembre 2020,2020-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.0,NaN,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,novembre 2020
1,2020-12,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71.0,NaN,NaN,41.0,NaN,NaN,NaN,31.0,1.0,NaN,131.0,NaN,NaN,52.0,NaN,NaN,NaN,39.0,55.0,49.0,NaN,1,décembre 2020,2020-112020-12,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71.0,NaN,NaN,41.0,NaN,NaN,NaN,54.0,1.0,NaN,144.0,NaN,NaN,52.0,NaN,NaN,NaN,39.0,55.0,49.0,NaN,1,novembre 2020décembre 2020
2,2021-01,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,3.0,NaN,1.0,NaN,10.0,NaN,NaN,17.0,NaN,NaN,4.0,NaN,NaN,NaN,12.0,3.0,2.0,1.0,2,janvier 2021,2020-112020-122021-01,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76.0,NaN,NaN,44.0,NaN,1.0,NaN,64.0,NaN,NaN,161.0,NaN,NaN,56.0,NaN,NaN,NaN,51.0,58.0,51.0,1.0,3,novembre 2020décembre 2020janvier 2021
3,2021-02,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,3.0,NaN,NaN,NaN,11.0,NaN,NaN,15.0,NaN,NaN,6.0,NaN,NaN,NaN,21.0,6.0,5.0,8.0,3,février 2021,2020-112020-122021-012021-02,15.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.0,NaN,NaN,47.0,NaN,NaN,NaN,75.0,NaN,NaN,176.0,NaN,NaN,62.0,NaN,NaN,NaN,72.0,64.0,56.0,9.0,6,novembre 2020décembre 2020janvier 2021février ...
4,2021-03,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,13.0,NaN,NaN,NaN,6.0,NaN,NaN,38.0,NaN,NaN,24.0,NaN,NaN,NaN,27.0,24.0,23.0,6.0,4,mars 2021,2020-112020-122021-012021-022021-03,NaN,3.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,87.0,NaN,NaN,60.0,NaN,NaN,NaN,81.0,NaN,NaN,214.0,NaN,NaN,86.0,NaN,NaN,NaN,99.0,88.0,79.0,15.0,10,novembre 2020décembre 2020janvier 2021février ...
5,2021-04,1.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,NaN,2.0,16.0,NaN,NaN,NaN,15.0,NaN,NaN,5.0,NaN,2.0,9.0,NaN,NaN,NaN,8.0,12.0,8.0,6.0,5,avril 2021,2020-112020-122021-012021-022021-032021-04,16.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,107.0,NaN,2.0,76.0,NaN,NaN,NaN,96.0,NaN,NaN,219.0,NaN,2.0,95.0,NaN,NaN,NaN,107.0,100.0,87.0,21.0,15,novembre 2020décembre 2020janvier 2021février ...
6,2021-05,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,NaN,1.0,NaN,NaN,NaN,6.0,NaN,NaN,6.0,NaN,2.0,1.0,NaN,NaN,2.0,2.0,6.0,7.0,3.0,6,mai 2021,2020-112020-122021-012021-022021-032021-042021-05,17.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,119.0,NaN,NaN,77.0,NaN,NaN,NaN,102.0,NaN,NaN,225.0,NaN,4.0,96.0,NaN,NaN,2.0,109.0,106.0,94.0,24.0,21,novembre 2020décembre 2020janvier 2021février ...
7,2021-06,NaN,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.0,NaN,NaN,24.0,NaN,3.0,NaN,15.0,NaN,NaN,44.0,NaN,25.0,24.0,NaN,NaN,6.0,28.0,22.0,39.0,12.0,7,juin 2021,2020-112020-122021-012021-022021-032021-042021...,NaN,27.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,146.0,NaN,NaN,101.0,NaN,4.0,NaN,117.0,NaN,NaN,269.0,NaN,29.0,120.0,NaN,NaN,8.0,137.0,128.0,133.0,36.0,28,novembre 2020décembre 2020janvier 2021février ...
8,2021-07,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,NaN,NaN,8.0,NaN,1.0,NaN,1.0,NaN,NaN,21.0,NaN,16.0,5.0,NaN,NaN,6.0,17.0,6.0,19.0,12.0,8,juillet 2021,2020-112020-122021-012021-022021-032021-042021...,NaN,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,162.0,NaN,NaN,109.0,NaN,5.0,NaN,118.0,NaN,NaN,290.0,NaN,45.0,125.0,NaN,NaN,14.0,154.0,134.0,152.0,48.0,36,novembre 2020décembre 2020janvier 2021février ...
9,2021-08,NaN,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,17.0,4.0,NaN,NaN,2.0,NaN,3.0,3.0,3.0,9,août 2021,2020-112020-122021-012021-022021-032021-042021...,NaN,47.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,167.0,NaN,NaN,110.0,NaN,NaN,NaN,NaN,NaN,NaN,293.

In [ ]:
comptePro.drop(columns=['index1','index1 cumulé', 'datePresentation cumulé', 'createdAt cumulé'], inplace=True)

In [ ]:
compteProCumul = comptePro[['06 cumulé', '13 cumulé','15 cumulé', '33 cumulé','34 cumulé', '36 cumulé', '44 cumulé', '56 cumulé', '59 cumulé', '67 cumulé', '69 cumulé', '75 cumulé', '76 cumulé', '77 cumulé',
                                         '78 cumulé', '91 cumulé', '92 cumulé', '93 cumulé','94 cumulé','95 cumulé']].fillna(method="ffill")

In [ ]:
comptePro.drop(columns=['06 cumulé', '13 cumulé','15 cumulé', '33 cumulé','34 cumulé', '36 cumulé', '44 cumulé', '56 cumulé', '59 cumulé', '67 cumulé', '69 cumulé', '75 cumulé', '76 cumulé', '77 cumulé',
                                         '78 cumulé', '91 cumulé', '92 cumulé', '93 cumulé','94 cumulé','95 cumulé'], inplace = True)
comptePro = comptePro.join(compteProCumul)

In [ ]:
comptePro["Général cumulé"] = comptePro[['06 cumulé', '13 cumulé','15 cumulé', '33 cumulé','34 cumulé', '36 cumulé', '44 cumulé', '56 cumulé', '59 cumulé', '67 cumulé', '69 cumulé', '75 cumulé', '76 cumulé', '77 cumulé',
                                         '78 cumulé', '91 cumulé', '92 cumulé', '93 cumulé','94 cumulé','95 cumulé']].sum(axis=1)
comptePro

territories,createdAt,,06,13,15,16,21,24,26,2A,33,34,36,44,56,59,63,67,69,7,75,76,77,78,83,87,91,92,93,94,95,datePresentation,cumulé,16 cumulé,21 cumulé,24 cumulé,26 cumulé,2A cumulé,63 cumulé,7 cumulé,83 cumulé,87 cumulé,06 cumulé,13 cumulé,15 cumulé,33 cumulé,34 cumulé,36 cumulé,44 cumulé,56 cumulé,59 cumulé,67 cumulé,69 cumulé,75 cumulé,76 cumulé,77 cumulé,78 cumulé,91 cumulé,92 cumulé,93 cumulé,94 cumulé,95 cumulé,Général cumulé
0,2020-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.0,NaN,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,novembre 2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.0
1,2020-12,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71.0,NaN,NaN,41.0,NaN,NaN,NaN,31.0,1.0,NaN,131.0,NaN,NaN,52.0,NaN,NaN,NaN,39.0,55.0,49.0,NaN,décembre 2020,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71.0,NaN,NaN,41.0,NaN,NaN,54.0,1.0,144.0,NaN,NaN,52.0,NaN,39.0,55.0,49.0,NaN,506.0
2,2021-01,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,3.0,NaN,1.0,NaN,10.0,NaN,NaN,17.0,NaN,NaN,4.0,NaN,NaN,NaN,12.0,3.0,2.0,1.0,janvier 2021,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76.0,NaN,NaN,44.0,NaN,1.0,64.0,1.0,161.0,NaN,NaN,56.0,NaN,51.0,58.0,51.0,1.0,564.0
3,2021-02,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,3.0,NaN,NaN,NaN,11.0,NaN,NaN,15.0,NaN,NaN,6.0,NaN,NaN,NaN,21.0,6.0,5.0,8.0,février 2021,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,83.0,NaN,NaN,47.0,NaN,1.0,75.0,1.0,176.0,NaN,NaN,62.0,NaN,72.0,64.0,56.0,9.0,648.0
4,2021-03,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,13.0,NaN,NaN,NaN,6.0,NaN,NaN,38.0,NaN,NaN,24.0,NaN,NaN,NaN,27.0,24.0,23.0,6.0,mars 2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1.0,NaN,87.0,NaN,NaN,60.0,NaN,1.0,81.0,1.0,214.0,NaN,NaN,86.0,NaN,99.0,88.0,79.0,15.0,815.0
5,2021-04,1.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,NaN,2.0,16.0,NaN,NaN,NaN,15.0,NaN,NaN,5.0,NaN,2.0,9.0,NaN,NaN,NaN,8.0,12.0,8.0,6.0,avril 2021,16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,1.0,NaN,107.0,NaN,2.0,76.0,NaN,1.0,96.0,1.0,219.0,NaN,2.0,95.0,NaN,107.0,100.0,87.0,21.0,922.0
6,2021-05,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,NaN,1.0,NaN,NaN,NaN,6.0,NaN,NaN,6.0,NaN,2.0,1.0,NaN,NaN,2.0,2.0,6.0,7.0,3.0,mai 2021,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,1.0,NaN,119.0,NaN,2.0,77.0,NaN,1.0,102.0,1.0,225.0,NaN,4.0,96.0,2.0,109.0,106.0,94.0,24.0,972.0
7,2021-06,NaN,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.0,NaN,NaN,24.0,NaN,3.0,NaN,15.0,NaN,NaN,44.0,NaN,25.0,24.0,NaN,NaN,6.0,28.0,22.0,39.0,12.0,juin 2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.0,1.0,NaN,146.0,NaN,2.0,101.0,NaN,4.0,117.0,1.0,269.0,NaN,29.0,120.0,8.0,137.0,128.0,133.0,36.0,1259.0
8,2021-07,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,NaN,NaN,8.0,NaN,1.0,NaN,1.0,NaN,NaN,21.0,NaN,16.0,5.0,NaN,NaN,6.0,17.0,6.0,19.0,12.0,juillet 2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.0,1.0,NaN,162.0,NaN,2.0,109.0,NaN,5.0,118.0,1.0,290.0,NaN,45.0,125.0,14.0,154.0,134.0,152.0,48.0,1395.0
9,2021-08,NaN,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,17.0,4.0,NaN,NaN,2.0,NaN,3.0,3.0,3.0,août 2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.0,1.0,NaN,167.0,NaN,2.0,110.0,NaN,5.0,118.0,1.0,293.0,NaN,62.0,129.0,16.0,154.0,137.0,155.0,51.0,1448.0


### E - Registration in the file of the *month*

In [ ]:
with open("{}/orga_figure3.csv".format(path), 'w') as f:  comptePro.to_csv(f)

# III - Display (figure 4)

### A - Importation

In [ ]:
! pip install airtable-python-wrapper #voir ici pour les méthodes de ce wrapper : https://airtable-python-wrapper.readthedocs.io/en/master/api.html

import json
import pandas as pd
import numpy as np
from airtable import Airtable
import requests


def download_airtable(table):
  #fichier avec les clés api de toutes les tables.
  cles = pd.read_csv("/content/drive/Shareddrives/Solinum/Mesure d'impact/Data/Fichiers utiles/TESTAPIKeyAirtable.csv", sep=";", dtype={"Département" :str})
  

  api_key= "key1NXKmwNUttHTmR" #clé api perso, à remplacer
  fiches = cles[cles["Nom de la table"] == table] # on choisit quel type de table on va télécharger (ex: les fiches, les hébergeurs, les organismes)

  df_list = []
  
  for index, row in fiches.iterrows(): 
      base_key = row["Clé base"]
      airtable = Airtable(base_key, table,api_key) #on sélectionne notre table 
      '''
      if table == "Fiches":
        print("c'est une fiches")
        soliguide = airtable.get_all(fields= ['Nom', 'Organisation', 'Etat','Validé', "Date màj"])
      if table == 'Diffusion':
        soliguide = airtable.get_all(fields= ["Nom de l'action", 'Type', 'Nb de pros', 'Nb de bénéficiaires', 'Date'])
      else:
        soliguide = airtable.get_all()
      '''
      soliguide = airtable.get_all() # on la télécharge en entier
      df = pd.DataFrame((r['fields'] for r in soliguide)) # on la transforme en dataframe
      df["territoire"] = row["Département"]
      print("territoire ",row["Département"], len(df), table)
      df_list.append(df)
      

      
  df = pd.concat(df_list,join='inner', ignore_index=True) #on concatène tous les départements
  df = df.apply(lambda x : x.astype(str).str.strip("[]").str.replace(", ",",").str.replace("'","")) #nettoyage des colonnes avec des listes
  return df

# c'est la fonction fournie par google ici : https://developers.google.com/analytics/devguides/reporting/core/v4/quickstart/service-py
from pandas.tseries.offsets import MonthEnd
import argparse
from apiclient.discovery import build
import httplib2
from oauth2client import client
from oauth2client import file
from oauth2client import tools
SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
DISCOVERY_URI = 'https://analyticsreporting.googleapis.com/$discovery/rest'

"""
This is the hard moment but don't worry only 6 steps
1) You need to go to https://console.cloud.google.com/
2)  Go in the left part in 'identifiants' 
3) Create a new 'identifiants'
be careful: Don't select "web application" but select "ordinateur de bureau" 
5) Import your client secrets in the google drive 
6) Write the client_secrets_path here ;)
"""

#CLIENT_SECRETS_PATH =  "/content/drive/Shareddrives/Solinum/Mesure d'impact/Data/Google Analytics/code_secret_client_870087239869-3hi36dis6rem74q06cgenvfumpdh9go9.apps.googleusercontent.com.json" # Path to client_secrets.json file.
CLIENT_SECRETS_PATH = "/content/drive/Shareddrives/Solinum/Mesure d'impact/Extraction python/Extraction Python/code_secret_client_954505251462-0doh6npvkqbbgpmefhmhgsnsdeamdf6f.apps.googleusercontent.com.json"
#/content/drive/Shareddrives/Solinum/Mesure d'impact/Extraction python/Extraction Python/secretclientNoemie12.json

def authenticate_ga_api():
  """Initializes the analyticsreporting service object.
  Returns:
    analytics an authorized analyticsreporting service object.
  """
  # Parse command-line arguments.
  parser = argparse.ArgumentParser(
      formatter_class=argparse.RawDescriptionHelpFormatter,
      parents=[tools.argparser])
  flags = parser.parse_args([])

  # Set up a Flow object to be used if we need to authenticate.
  flow = client.flow_from_clientsecrets(
      CLIENT_SECRETS_PATH, scope=SCOPES,
      message=tools.message_if_missing(CLIENT_SECRETS_PATH))

  '''Prepare credentials, and authorize HTTP object with them.
  If the credentials don't exist or are invalid run through the native client
  flow. The Storage object will ensure that if successful the good
  credentials will get written back to a file. '''
  
  storage = file.Storage('analyticsreporting.dat')
  credentials = storage.get()
  if credentials is None or credentials.invalid:
    credentials = tools.run_flow(flow, storage, flags)
  http = credentials.authorize(http=httplib2.Http())

  # Build the service object.
  analytics = build('analytics', 'v4', http=http, discoveryServiceUrl=DISCOVERY_URI)

  return analytics

# transforme le json donné par ga en dataframe
def analytics__todataframe():
  data_dic = {f"{i}": [] for i in DIMS + METRICS}
  for report in data.get('reports', []):
      rows = report.get('data', {}).get('rows', [])
      for row in rows:
          for i, key in enumerate(DIMS):
              data_dic[key].append(row.get('dimensions', [])[i]) # Get dimensions
          dateRangeValues = row.get('metrics', [])
          for values in dateRangeValues:
              all_values = values.get('values', []) # Get metric values
              for i, key in enumerate(METRICS):
                  data_dic[key].append(all_values[i])


  df = pd.DataFrame(data=data_dic)
  df.columns = [col.split(':')[-1] for col in df.columns]
  return df



In [ ]:
SCOPES

['https://www.googleapis.com/auth/analytics.readonly']

### B - Selection of date

In [ ]:
# You have to add at the end of year 2021 the date that you want to stop (for automatisation put datetime.now())
year2022 = ['2022-01-01','2022-02-01', '2022-03-01', '2022-04-01', '2022-05-01', '2022-06-01','2022-07-01', '2022-08-01', '2022-09-01', '2022-10-01','2022-11-01','2022-12-01']
year2021 = ['2021-01-01','2021-02-01', '2021-03-01', '2021-04-01', '2021-05-01', '2021-06-01','2021-07-01', '2021-08-01', '2021-09-01', '2021-10-01','2021-11-01','2021-12-01']
year2020 = ['2020-01-01', '2020-02-01', '2020-03-01','2020-04-01', '2020-05-01','2020-06-01', '2020-07-01','2020-08-01','2020-09-01', '2020-10-01', '2020-11-01', '2020-12-01']
year2019 = ['2019-01-01','2019-02-01','2019-03-01','2019-04-01', '2019-05-01','2019-06-01', '2019-07-01', '2019-08-01','2019-09-01','2019-10-01','2019-11-01','2019-12-01']
year2018 = ['2018-07-01', '2018-08-01','2018-09-01','2018-10-01','2018-11-01','2018-12-01']
chosenDate = year2018 + year2019 + year2020 + year2021 + year2022

### C - Authentification GA

In [ ]:
"""
Omg an other hard moment but only 4 steps ;)
1) Ask to Victoria all access to segments of google analytics
When she send it:

-> It open a page with "view" "name"
2) Select the view : Soliguide/Toutes les donnees du site web
3) Click on OK
It open a page with a URL
3) Then look at the url. For exemple (1) : https://analytics.google.com/analytics/web/?authuser=0#/a111184026w165785052p166309162/admin/segments/m-content.mode=EDIT&m-content.segmentId=uservTdpLhQxTTCdGm7RZ0Gbtw
4) Select the last part after 'user'. For exemple (1) : 93IiWmGTSB-LJsU_cCgejQ
"""
departement06 = "93IiWmGTSB-LJsU_cCgejQ"
departement07 = "5HuXYH4BTq2gt-xyVTqXlw"
departement13 = "0G4OcLGwQ4OcWvkf4XnRSQ"
departement15 = "72M7BBOqTKKLNmNbL2449A"
departement16 = "np3CJWPqRHOnPznK94e1yg"
departement21 = "Nc9uDhVKQWSLf0iTy9CBcA"
departement26 = "G_PUzVuLTiCzZvGh0tcasQ"
departement33 = "SrOly7cJQHu0fs7GFUmqGw"
departement34 = "2sTOYOJSTl274PoTG59_QQ"
departement35 = "Zm_nPn_wSKqR-5zWaTYolw"
departement36 = "_VPbvaosRD-1Of4KVfHb_A"
departement44 = "AjDC4DEBShqSx6_kdvJgUA"
departement56 = "lT7nsWGlSn2gZidjlgD99w"
departement59 = "k0NmjmQCRgeM9o78SwkNoA"
departement63 = "BAk252tGQKO0yNVxK6xtbA"
departement67 = "RL6mSTHGT_-wrOr7SzTRsA"
departement75 = "dgx0rh3DTcKtlzMQepx2Pg"
#departement75 = "dgx0rh3DTcKtlzMQepx2Pg"
departement76 = "3aQMoDbFRHOwTvnc3xUaTQ"
departement77 = "w1Y1ZcuOSIiM0zIixDcVcw"
departement78 = "1x793o2hRGC3JLmrVuIOzg"
departement91 = "KeUBbwiJSGWkBRirV2j04g"
departement92 = "9Nq2J6DGQRGULz3WM8LTfA"
departement93 = "NSODhTIVQFeiGodF_th-4g"
departement94 = "dkipAMTWSCmeeuhLxrc31A"
departement95 = "m6D1BEjxRJOsd78KcA81Hw"
#departmentAIE = [33, 44, 67,75,78,92,93,94, 95, 6, 77,91]
SEGMENTS = [departement06, departement07, departement13, departement15, departement21, departement26, departement33, departement34, departement35, departement36, 
            departement44, departement56, departement59, departement63, departement67, departement75, departement76, departement77, departement78, departement91, departement92, 
            departement93, departement94, departement95]
print(SEGMENTS)

ga_auth = authenticate_ga_api()



VIEW_ID = "166309162"
DIMS = ['ga:segment']
METRICS = ['ga:users',"ga:sessions","ga:pageviews"]

['93IiWmGTSB-LJsU_cCgejQ', '5HuXYH4BTq2gt-xyVTqXlw', '0G4OcLGwQ4OcWvkf4XnRSQ', '72M7BBOqTKKLNmNbL2449A', 'Nc9uDhVKQWSLf0iTy9CBcA', 'G_PUzVuLTiCzZvGh0tcasQ', 'SrOly7cJQHu0fs7GFUmqGw', '2sTOYOJSTl274PoTG59_QQ', 'Zm_nPn_wSKqR-5zWaTYolw', '_VPbvaosRD-1Of4KVfHb_A', 'AjDC4DEBShqSx6_kdvJgUA', 'lT7nsWGlSn2gZidjlgD99w', 'k0NmjmQCRgeM9o78SwkNoA', 'BAk252tGQKO0yNVxK6xtbA', 'RL6mSTHGT_-wrOr7SzTRsA', 'dgx0rh3DTcKtlzMQepx2Pg', '3aQMoDbFRHOwTvnc3xUaTQ', 'w1Y1ZcuOSIiM0zIixDcVcw', '1x793o2hRGC3JLmrVuIOzg', 'KeUBbwiJSGWkBRirV2j04g', '9Nq2J6DGQRGULz3WM8LTfA', 'NSODhTIVQFeiGodF_th-4g', 'dkipAMTWSCmeeuhLxrc31A', 'm6D1BEjxRJOsd78KcA81Hw']


In [ ]:
ga_auth = authenticate_ga_api() 


VIEW_ID = "166309162"
DIMS = ['ga:segment']
METRICS = ['ga:users',"ga:sessions","ga:pageviews"]

### D - Download from GA

In [ ]:
dic_ga = {}
for indiceChosenDatei in range(1, len(chosenDate)):
  df_list = []
  for segment in SEGMENTS:
    print(segment)
    requests_list =  [{
            'viewId': VIEW_ID,
            'pageSize': 20000,  
            'dateRanges': [{'startDate': chosenDate[indiceChosenDatei-1], 'endDate': chosenDate[indiceChosenDatei]}],
            'dimensions': [{'name': name} for name in DIMS],
            'metrics': [{'expression': exp} for exp in METRICS],
            'segments' : [{"segmentId" : "gaid::{}".format(segment)}]
    }]
    data = ga_auth.reports().batchGet(body={'reportRequests':requests_list }).execute()
    df = analytics__todataframe()
    df_list.append(df)
  stats_ga = pd.concat(df_list).rename(columns={"segment" : "territoire", "users" : "Utilisateurs","sessions" : "Sessions","pageviews" : "Pages vues"}).set_index("territoire",drop=True).astype("int64")
  dic_ga['du {} au {}'.format(chosenDate[indiceChosenDatei-1], chosenDate[indiceChosenDatei])] = stats_ga

93IiWmGTSB-LJsU_cCgejQ
5HuXYH4BTq2gt-xyVTqXlw
0G4OcLGwQ4OcWvkf4XnRSQ
72M7BBOqTKKLNmNbL2449A
Nc9uDhVKQWSLf0iTy9CBcA
G_PUzVuLTiCzZvGh0tcasQ
SrOly7cJQHu0fs7GFUmqGw
2sTOYOJSTl274PoTG59_QQ
Zm_nPn_wSKqR-5zWaTYolw
_VPbvaosRD-1Of4KVfHb_A
AjDC4DEBShqSx6_kdvJgUA
lT7nsWGlSn2gZidjlgD99w
k0NmjmQCRgeM9o78SwkNoA
BAk252tGQKO0yNVxK6xtbA
RL6mSTHGT_-wrOr7SzTRsA
dgx0rh3DTcKtlzMQepx2Pg
3aQMoDbFRHOwTvnc3xUaTQ
w1Y1ZcuOSIiM0zIixDcVcw
1x793o2hRGC3JLmrVuIOzg
KeUBbwiJSGWkBRirV2j04g
9Nq2J6DGQRGULz3WM8LTfA
NSODhTIVQFeiGodF_th-4g
dkipAMTWSCmeeuhLxrc31A
m6D1BEjxRJOsd78KcA81Hw
93IiWmGTSB-LJsU_cCgejQ
5HuXYH4BTq2gt-xyVTqXlw
0G4OcLGwQ4OcWvkf4XnRSQ
72M7BBOqTKKLNmNbL2449A
Nc9uDhVKQWSLf0iTy9CBcA
G_PUzVuLTiCzZvGh0tcasQ
SrOly7cJQHu0fs7GFUmqGw
2sTOYOJSTl274PoTG59_QQ
Zm_nPn_wSKqR-5zWaTYolw
_VPbvaosRD-1Of4KVfHb_A
AjDC4DEBShqSx6_kdvJgUA
lT7nsWGlSn2gZidjlgD99w
k0NmjmQCRgeM9o78SwkNoA
BAk252tGQKO0yNVxK6xtbA
RL6mSTHGT_-wrOr7SzTRsA
dgx0rh3DTcKtlzMQepx2Pg
3aQMoDbFRHOwTvnc3xUaTQ
w1Y1ZcuOSIiM0zIixDcVcw
1x793o2hRGC3JLmrVuIOzg
KeUBbwiJSGW

### E - Creation of a dataFrame to Display

In [ ]:
def ecrireGA(chosenDate, nbDep):
  mois ={}
  for indiceChosenDatei in range(1, len(chosenDate)):
    date_intervalle = 'du {} au {}'.format(chosenDate[indiceChosenDatei-1], chosenDate[indiceChosenDatei])
    if nbDep != 'Paris+ivry': 
      mois[date_intervalle] = dic_ga[date_intervalle][dic_ga[date_intervalle].index == 'Département {}'.format(nbDep)]
    else:
      mois[date_intervalle] = dic_ga[date_intervalle][dic_ga[date_intervalle].index == 'Paris+ivry']
  moisGA = pd.concat(mois, axis=0)
  return(moisGA)


# Here I selected data from 
alldep = ['06', '07', '13', '15', '16', '21', '24', '33', '34','35', '36', '44', '56', '59', '63', '67','Paris+ivry', '76', '77', '78', '87', '91','92','93','94', '95']
L = []
for dep in alldep:
  L.append(ecrireGA(chosenDate, dep))

pd.concat(L, axis=0)

,,Utilisateurs,Sessions,Pages vues
,territoire,,,
du 2020-01-01 au 2020-02-01,Département 06,8,15,82
du 2020-02-01 au 2020-03-01,Département 06,10,26,201
du 2020-03-01 au 2020-04-01,Département 06,41,49,203
du 2020-04-01 au 2020-05-01,Département 06,52,86,587
du 2020-05-01 au 2020-06-01,Département 06,40,95,553
...,...,...,...,...
du 2021-11-01 au 2021-12-01,Département 95,173,216,478
du 2021-12-01 au 2022-01-01,Département 95,158,214,588
du 2022-01-01 au 2022-02-01,Département 95,237,311,616


In [ ]:
df_recherches = pd.concat(L, axis=0)
df_recherches #Paris+ivry	
d = {'Paris+ivry':'Département 75'}
idx = df_recherches.index.levels[1].to_series().replace(d)
df_recherches.index = df_recherches.index.set_levels(idx, level=1)
df_recherches

,,Utilisateurs,Sessions,Pages vues
,territoire,,,
du 2020-01-01 au 2020-02-01,Département 06,8,15,82
du 2020-02-01 au 2020-03-01,Département 06,10,26,201
du 2020-03-01 au 2020-04-01,Département 06,41,49,203
du 2020-04-01 au 2020-05-01,Département 06,52,86,587
du 2020-05-01 au 2020-06-01,Département 06,40,95,553
...,...,...,...,...
du 2021-11-01 au 2021-12-01,Département 95,173,216,478
du 2021-12-01 au 2022-01-01,Département 95,158,214,588
du 2022-01-01 au 2022-02-01,Département 95,237,311,616


### F - Registration in the file of the *month*

In [ ]:
with open("{}/GAdata.csv".format(path), 'w') as f: df_recherches.to_csv(f)

# IV -  Display map (figure 5)

### A - Importing files

In [ ]:
!pip install geojson
import pandas as pd
import numpy as np
from geojson import Feature, Point, FeatureCollection
import geojson
import json

### B - Importing mapping files

In [ ]:
dep = pd.read_csv("/content/drive/Shareddrives/Solinum/Mesure d'impact/Soliguide/population par territoire/correspondance-code-insee-code-postal.csv", sep=";")
dep["geo_shape"] = dep["geo_shape"].apply(lambda x: json.loads(x))

In [ ]:
dep = pd.read_csv("/content/drive/Shareddrives/Solinum/Mesure d'impact/Soliguide/population par territoire/correspondance-code-insee-code-postal.csv", sep=";")
dep

,Code INSEE,Code Postal,Commune,Département,Région,Statut,Altitude Moyenne,Superficie,Population,geo_point_2d,geo_shape,ID Geofla,Code Commune,Code Canton,Code Arrondissement,Code Département,Code Région
0,59416,59190,MORBECQUE,NORD,NORD-PAS-DE-CALAIS,Commune simple,23.0,4455.0,2.7,"50.6768934286,2.53621614433","{""type"": ""Polygon"", ""coordinates"": [[[2.501239...",1237,416,30,4,59,31
1,22102,22330,LANGOURLA,COTES-D'ARMOR,BRETAGNE,Commune simple,170.0,2165.0,0.6,"48.2846411077,-2.41550101132","{""type"": ""Polygon"", ""coordinates"": [[[-2.41629...",10183,102,9,1,22,53
2,31225,31310,GOUTEVERNISSE,HAUTE-GARONNE,MIDI-PYRENEES,Commune simple,264.0,485.0,0.2,"43.2140263014,1.17376592029","{""type"": ""Polygon"", ""coordinates"": [[[1.165580...",20555,225,27,1,31,73
3,23025,23220,BONNAT,CREUSE,LIMOUSIN,Chef-lieu canton,355.0,4554.0,1.3,"46.3230193342,1.91345063136","{""type"": ""Polygon"", ""coordinates"": [[[1.952331...",3586,25,6,2,23,74
4,38522,38740,VALJOUFFREY,ISERE,RHONE-ALPES,Commune simple,2009.0,12644.0,0.1,"44.8815349113,6.07950399615","{""type"": ""Polygon"", ""coordinates"": [[[6.056489...",1082,522,36,1,38,82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36737,84137,84110,VAISON-LA-ROMAINE,VAUCLUSE,PROVENCE-ALPES-COTE D'AZUR,Chef-lieu canton,248.0,2710.0,6.2,"44.247701867,5.06040782309","{""type"": ""Polygon"", ""coordinates"": [[[5.060410...",2642,137,21,3,84,93
36738,41248,41170,SOUDAY,LOIR-ET-CHER,CENTRE,Commune simple,158.0,3649.0,0.5,"48.0447092121,0.846594759389","{""type"": ""Polygon"", ""coordinates"": [[[0.869380...",614,248,11,2,41,24
36739,21194,21700,CORGOLOIN,COTE-D'OR,BOURGOGNE,Commune simple,222.0,1262.0,0.9,"47.0819087095,4.93531164133","{""type"": ""Polygon"", ""coordinates"": [[[4.962715...",19373,194,24,1,21,26
36740,60605,60210,SARNOIS,OISE,PICARDIE,Commune simple,193.0,555.0,0.3,"49.680399425,1.92482666497","{""type"": ""Polygon"", ""coordinates"": [[[1.909369...",27556,605,17,1,60,22


In [ ]:
dep = pd.read_csv("/content/drive/Shareddrives/Solinum/Mesure d'impact/Soliguide/population par territoire/correspondance-code-insee-code-postal.csv", sep=";")
dep["geo_shape"] = dep["geo_shape"].apply(lambda x: json.loads(x))
L = []
for index in dep.index:
  L.append(Feature(geometry=dep["geo_shape"][index],properties={"code": dep["Code Postal"][index]}))
dep["geo_shape_code"] = L
data2 = FeatureCollection(list(dep["geo_shape_code"]))

### C - Mapping with our files

In [ ]:
fichesvc = pd.DataFrame(fichesJSON["codePostal"].value_counts())
fichesvc["location"] = fichesvc.index
fichesvc.reset_index()

,index,codePostal,location
0,75018,272,75018
1,75020,259,75020
2,75013,247,75013
3,75019,229,75019
4,33000,211,33000
...,...,...,...
1807,71270,1,71270
1808,26700,1,26700
1809,23400,1,23400
1810,23300,1,23300


In [ ]:
fichesvc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1812 entries, 75018 to 24120
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   codePostal  1812 non-null   int64 
 1   location    1812 non-null   object
dtypes: int64(1), object(1)
memory usage: 42.5+ KB


In [ ]:
fichesvc = pd.merge(fichesvc, dep[['Code Postal','Commune']], how='left', left_on='location', right_on='Code Postal')
fichesvc

,codePostal,location,Code Postal,Commune
0,272,75018,75018,PARIS-18E-ARRONDISSEMENT
1,259,75020,75020,PARIS-20E-ARRONDISSEMENT
2,247,75013,75013,PARIS-13E-ARRONDISSEMENT
3,229,75019,75019,PARIS-19E-ARRONDISSEMENT
4,211,33000,NaN,NaN
...,...,...,...,...
9643,1,24120,24120,VILLAC
9644,1,24120,24120,LA FEUILLADE
9645,1,24120,24120,CHAVAGNAC
9646,1,24120,24120,BEAUREGARD-DE-TERRASSON


### Be careful to postal code writing like this : instead of this: 75001 you have this: 75001/75002/75003

In [ ]:
fichesvc['dep'] = fichesvc.location.str[0:2]
fichesvc

,codePostal,location,Code Postal,Commune,dep
0,272,75018,75018,PARIS-18E-ARRONDISSEMENT,75
1,259,75020,75020,PARIS-20E-ARRONDISSEMENT,75
2,247,75013,75013,PARIS-13E-ARRONDISSEMENT,75
3,229,75019,75019,PARIS-19E-ARRONDISSEMENT,75
4,211,33000,NaN,NaN,33
...,...,...,...,...,...
9643,1,24120,24120,VILLAC,24
9644,1,24120,24120,LA FEUILLADE,24
9645,1,24120,24120,CHAVAGNAC,24
9646,1,24120,24120,BEAUREGARD-DE-TERRASSON,24


In [ ]:
with open("{}/fichesvc.csv".format(path), 'w') as f:  fichesvc.to_csv(f)

### D - Show map

In [ ]:
import folium
def folium_del_legend(choropleth: folium.Choropleth):
  """A hack to remove choropleth legends.
  
  The choropleth color-scaled legend sometimes looks too crowded. Until there is an
  option to disable the legend, use this routine to remove any color map children
  from the choropleth.
  
  Args:
    choropleth: Choropleth objected created by `folium.Choropleth()`
    
  Returns:
    The same object `choropleth` with any child whose name starts with
    'color_map' removed.
  """
  del_list = []
  for child in choropleth._children:
    if child.startswith('color_map'):
      del_list.append(child)
  for del_item in del_list:
    choropleth._children.pop(del_item)
  return choropleth

#m = folium.Map(...)
#folium_del_legend(
#    folium.Choropleth(...)).add_to(m)

In [ ]:
from folium.plugins import FloatImage

france = folium.Map(location = [46.232192999999995, 2.209666999999996], zoom_start = 6)
folium_del_legend(folium.Choropleth(data2, 
                  key_on = "feature.properties.code",
                  data = fichesvc.dropna(), 
                  columns = ["location","codePostal",'Commune'], 
                  fill_color='YlGnBu', 
                  fill_opacity=1,
                  line_opacity=0.4,
                  legend_name="Structures répertoriées (en ligne et brouillon) sur Soliguide.fr",
                  smooth_factor=0,
                  Highlight= True,
                  line_color = "#e5e5e5",
                  name = "Structures répertoriées sur Soliguide.fr",
                  show=False,
                  overlay=True,
                  nan_fill_color = "White",
                  popup=(fichesvc['codePostal']),
                  threshold_scale = [1,2,3,5,7,9,18,89,286]
                  )).add_to(france)


url = ('https://raw.githubusercontent.com/Antoine-Carre/bdd_python/main/data_csv/Scale_nber_of_fiches.png')
FloatImage(url, bottom=5, left=5).add_to(france)

france
#threshold_scale = [1,2,3,5,7,9,18,89,435]

### E - Save map in an folder

In [ ]:
france.save("{}/map.html".format(path))

# V - Display figure 2 

### This step is usefull to match postal code and a city so I used a base in metabase which contain help for it 

In [ ]:
# Ce fichier provient de la base "local" de metabase

local = pd.read_json("/content/drive/Shareddrives/Solinum/Mesure d'impact/analyse code/metabase/20211030/location.json", lines=True)

In [ ]:
r = re.compile("[0-9]{5}")


departement_location = []
nomVille_Code = {'bastille-paris': '75011',
 'bonnieres-sur-seine': '78270',
 'saint-maur-des-fosses':'94100',
 'cergy-pontoise': '95000',
 'elancourt': '78208',
 'evry-courcouronnes': '91000',
 'le-bourget':'93350',
'les-ulis':'91940',
'les-lilas':'93260',
'le-mans':'72000',
'la-rochelle':'17000',
 'gare-st-jean-bordeaux': '33000',
 'la-defense': '92400',
 'le-kremlin-bicetre': '94270', "paris":'75000', "courbevoie":"92400", "trappes": "78000", "val-d-oise":"95000", "la-courneuve": "93027", "val-oise":"95000", "limeil-brevannes": 	"94450", "la-chapelle-paris":"75000"}
for index in local.index:
  if local["geoType"][index] == "ville":
    code = r.findall(local["label"][index])
    if len(code) != 0:
      nomVille_Code[local["geoValue"][index]] = str(code[0])
  if local["geoType"][index] == "departement":
    departement_location.append(str(local["geoValue"][index]))

In [ ]:
for index in fichesJSON.index:
  nomVille_Code[fichesJSON["ville"][index].lower()] = str(fichesJSON["codePostal"][index])
  nomVille_Code[fichesJSON["departement"][index].lower()] = str(fichesJSON["codePostal"][index])

In [ ]:
# Creating the geoValue function that will give us the postal code from the city name
def geoValue(x):
  
  if type(x) == str:
    x = x.lower()
    if x in nomVille_Code.keys():
      return(str(nomVille_Code[x]))
    else:
      if len(r.findall(x)) != 0:
        return(str(r.findall(x)[0]))
      else:
        return(np.nan)
  else:
    return(np.nan)

In [ ]:
# Loading the search collection
s = pd.read_json("/content/drive/Shareddrives/Solinum/Mesure d'impact/analyse code/metabase/20220301/search.json", lines=True)

In [ ]:
# Creating feature for the geovalue, the postalcode and the 2 digit of each departement
s["GeoValue"] = s["location"].apply(lambda x: x["geoValue"])
s["Département"] = s["GeoValue"].apply(lambda x: geoValue(x))
s["Territoire"] = s["Département"].apply(lambda x: x[0:2] if type(x)!=float else x)

In [ ]:
dep = pd.read_csv("/content/drive/Shareddrives/Solinum/Mesure d'impact/Soliguide/population par territoire/correspondance-code-insee-code-postal.csv", sep=";")
dict_ville = pd.Series(dep[['Code Postal','Commune']].Commune.values,index=dep['Code Postal']).to_dict()
dict_ville['06000'] = "NICE"
dict_ville['13000'] = "MARSEILLE"
dict_ville['33000'] = "BORDEAUX"
dict_ville['34000'] = "MONTPELLIER"
dict_ville['35000'] = "RENNES"
dict_ville['44000'] = "NANTES"
dict_ville['59000'] = "LILLE"
dict_ville['63000'] = "CLERMONT-FERRAND"
dict_ville['67000'] = "STRASBOURG"
dict_ville['75000'] = "PARIS"
dict_ville['76000'] = "ROUEN"
dict_ville['95000'] = "CERGY"
dict_ville['77000'] = "MEULIN"
dict_ville['95000'] = "CERGY"

In [ ]:
s

In [ ]:
# Converting categorie code number into there written names

cat = {
  0: "Unknowed",
  100: "Santé",
  101: "Addiction",
  102: "Dépistage",
  103: "Psychologie",
  104: "Soins enfants",
  105: "Généraliste",
  106: "Dentaire",
  107: "Suivi grossesse",
  109: "Vaccination",
  110: "Infirmerie",
  111: "Vétérinaire",

  200: "Formation et emploi",
  201: "Atelier numérique",
  202: "Formation français",
  203: "Accompagnement à l'emploi",
  205: "Insertion par l'activité économique",
  206: "Soutien scolaire",

  300: "Hygiène et bien-être",
  301: "Douche",
  302: "Laverie",
  303: "Bien-être",
  304: "Toilettes",
  305: "Protections périodiques",
  306: "Masques",

  400: "Conseil",
  401: "Conseil logement",
  402: "Permanence juridique",
  403: "Domiciliation",
  404: "Accompagnement social",
  405: "Ecrivain public",
  406: "Conseil handicap",
  407: "Conseil administratif",
  408: "Conseil aux parents",

  500: "Technologie",
  501: "Ordinateur",
  502: "Wifi",
  503: "Prise",
  504: "Téléphones",
  505: "Coffre-fort numérique",

  600: "Alimentation",
  601: "Distribution alimentaire",
  602: "Restauration assise",
  603: "Colis Alimentaire",
  604: "Epicerie Sociale",
  605: "Fontaine à eau",

  700: "Accueil",
  701: "Accueil de jour",
  702: "Hébergement d'urgence",
  703: "Hébergement à long terme",
  704 : "Logement bas prix",
  705: "Espaces de repos",
  706: "Halte de nuit",
  707: "Garde d'enfants",
  708: "Espace famille",

  800: "Activités",
  801: "Sport",
  802: "Musée",
  803: "Bibliothèque",
  804: "Activités",
  805: "Animations et loisirs",

  900: "Matériel",
  901: "Bagagerie",
  902: "Magasin solidaire",
  903: "Vêtements",
  904: "Animaux",

  1100: "Spécialistes",
  1101: "Allergologie",
  1102: "Cardiologie",
  1103: "Dermatologie",
  1104: "Echographie",
  1105: "Endocrinologie",
  1106: "Gastro-entérologie",
  1107: "Gynécologie",
  1108: "Kinésithérapie",
  1109: "Mammographie",
  1110: "Ophtalmologie",
  1111: "Oto-rhino-laryngologie",
  1112: "Nutrition",
  1113: "Pédicure",
  1114: "Phlébologie",
  1115: "Pneumologie",
  1116: "Radiologie",
  1117: "Rhumatologie",
  1118: "Urologie",
  1119: "Orthophonie",
  1120: "Stomatologie",
  1121: "Osthéopathie",
  1122: "Accupuncture",

	1200: "Transport & mobilité",
  1201: "Co-voiturage",
  1202: "Mise à disposition de véhicule",
  1203: "Transport avec chauffeur",
  1204: "Aide au permis de conduire",
}

In [ ]:
# Formating the ccreatedAt column fro json to pandas object
s["createdAt"] = pd.json_normalize(s['createdAt'])

In [ ]:
# Changing the name of the categories from numerical code to there definition
s["categorie"] = s["categorie"].apply(lambda x:cat[x] if str(x)!="nan" else x)

# Formating value in the createdAt column datetime
s["Created At"] = pd.to_datetime(s["createdAt"])

# Creating a variable that will focus only in the month and year (putting the day always as 1)
s["year-month"] = s["Created At"].apply(lambda x: datetime(x.year, x.month, 1))


In [ ]:
s['userDatas'] = s['userDatas'].apply(lambda x: {} if pd.isna(x) else x)
s = s.join(pd.json_normalize(s.userDatas))

In [ ]:
s = s[s.status != 'ADMIN_SOLIGUIDE']

In [ ]:
s.info()

In [ ]:
s_cities = s[['year-month', 'Département','_id']]
s_cities


In [ ]:
df_recherches_per_city = s_cities.pivot_table(index=['Département'], columns='year-month', aggfunc='count')

df_recherches_per_city.reset_index(inplace=True)
df_recherches_per_city.columns = [f'{j}' for i, j in df_recherches_per_city.columns]
df_recherches_per_city.columns = df_recherches_per_city.columns.str.replace('-01 00:00:00', '')
df_recherches_per_city.rename(columns={"":"Code_Postal"}, inplace=True)

# df_recherches_per_city = df_recherches_per_city[['Code_Postal', 'Commune',	'2021-03',	'2021-04',	'2021-05',	'2021-06',	'2021-07',	'2021-08',	'2021-09',	'2021-10']]

df_recherches_per_city.columns.values[0] = "Code_Postal"
df_recherches_per_city

In [ ]:
L = []
#for n in range(len(df_recherches_per_city['Code_Postal'])):
for n in range(len(df_recherches_per_city.Code_Postal)):
  if df_recherches_per_city['Code_Postal'][n] in dict_ville:
    L.append(dict_ville[df_recherches_per_city['Code_Postal'][n]])
  else:
    L.append('')


df_recherches_per_city['Ville'] = L

In [ ]:
with open("{}/liste_search_per-city.csv".format(path), 'w') as f: df_recherches_per_city.to_csv(f)

### Creation of a dataFrame, in index = dates and in columns all categorie per departments

In [ ]:
def search_engine(df, dep, argue=""):
  dff = pd.crosstab( df["year-month"], df["categorie"] )
  L = []
  for col in dff: 
    if argue == "":
      dff = dff.rename(columns={col: "{} dep({})".format(col, dep)})
    else:
       dff = dff.rename(columns={col: "{} {}".format(col, "général")})
  return(dff)


  
L = [search_engine(s, "général", argue="général")]
TerritoireSoliguide = ['06','07','13','15','16','21','24', '33','34','35','36', '44', '56', '59','63', '67', '75','76','77', '78', '87', '91', '92', '93', '94', '95']

# Ajout de toutes les recherches sans spécification de catégorie
for dep in TerritoireSoliguide:
  L.append(pd.DataFrame(s[s["Territoire"] == dep]["year-month"].value_counts()).rename(columns={"year-month": "Recherches (dep {})".format(dep)}))
L.append(pd.DataFrame(s["year-month"].value_counts()).rename(columns={"year-month": "Recherches général"}))

# Ajout des recherhces par catégorie par département
for dep in TerritoireSoliguide:
  L.append(search_engine(s[s["Territoire"] == dep], dep, argue=""))

search = pd.concat(L, axis=1)
search["year-month"] = search.index
#search = datePresentation(search)

In [ ]:
search

In [ ]:
# Changing the name from year-month to match the name from previous dataset (sAVANT)
search['datePresentation'] = search['year-month'].astype(str)

# Standardizing the wording use in column names for each departement
search.columns = search.columns.str.replace('\(dep ', 'dep(')

In [ ]:
# Tiding up the wording for each departement in the previous dataset as well
sAVANT.columns = sAVANT.columns.str.replace('\(dep:', 'dep(')

In [ ]:
# Concatenatng the previous dataset and the new one
snew = pd.concat([sAVANT,search], axis='index')
snew.drop(columns=['Unnamed: 0','year-month'], inplace=True)

In [ ]:
# Cleaning up a bit the concatenated dataset
snew.reset_index(inplace=True)
snew.drop(columns='index', inplace=True)

In [ ]:
# Deleting value for the month of march 2021 that were impacted by our work on the database back then
snew.iloc[19] = 0
snew = snew.replace({0:np.nan})

In [ ]:
# Creating a row for the month of february 202& that was missing for the same reason as the previous cell
new_row = {'datePresentation':'2021-02-01'}

# Putting back the value in the datePresentation cell for the month of march 2021
snew.iloc[19,0] = '2021-03-01'

# Adding the new row for the month of february 2021 into the dataset
snew = snew.append(new_row, ignore_index=True)

# sorting the value the make sure the month are in a chronological order
snew = snew.sort_values(by=['datePresentation'])

In [ ]:
snew.reset_index(inplace=True)
snew = snew.iloc[:,1:]
snew

In [ ]:
# Filling up missing data from the 2 empty month with value extrapolated from the value in there respective columns
snew[17:22] = snew[17:22].interpolate(method ='linear', limit_direction ='forward')

In [ ]:
snew_2021 = snew.iloc[18:,:]
snew_2021

In [ ]:
test = snew.iloc[1:].cumsum()
test.rename(columns={'datePresentation':'datePresentation2'}, inplace=True)
snew[['datePresentation']].join(test)

### To registrer the dataFrame

In [ ]:
with open("{}/searchWithDatePresentation3.csv".format(path), 'w') as f: snew_2021.to_csv(f)

# VI-  Display 6

In [ ]:
! pip install airtable-python-wrapper #voir ici pour les méthodes de ce wrapper : https://airtable-python-wrapper.readthedocs.io/en/master/api.html

import json
import pandas as pd
import numpy as np
from airtable import Airtable
import requests
api_key= "keyzb3zm1wEQZvsfA" #clé api perso, à remplacer
base_key = "appsLJJwfdueN9xqD"
table = "Diffusion"
airtable = Airtable(base_key, table,api_key) #on sélectionne notre table
soliguide = airtable.get_all() # on la télécharge en entier
diffusion= pd.DataFrame((r['fields'] for r in soliguide))

In [ ]:
diffusion["Territoire"] = diffusion["Territoire"].apply(lambda x: x[0] if type(x)==list else "")

In [ ]:
from datetime import datetime
diffusion["Date date"] = diffusion["Date"].apply(lambda x: datetime.strptime(x, "%Y-%m-%d") if type(x) == str else np.nan )

In [ ]:
diffusion["year-month"] = diffusion["Date date"].apply(lambda x: str(x.year) + "-" + str(x.month) if type(x)!= float else x)

In [ ]:
def semestre_diffusion(diffusion, titre, semestre="maintenant"):
  month_pro_benef = dict()
  month_pro_benef_percent = dict()
  for date in np.unique(diffusion["year-month"]):
    if semestre == "ancien": 
      if ("2021-3" in date) or ("2021-4" in date) or ("2021-5" in date): 
        benef = diffusion[diffusion["year-month"] == date]['Nb de bénéficiaires'].sum()
        pro = diffusion[diffusion["year-month"] == date]["Nb de pros"].sum()
        tous = benef + pro
        month_pro_benef[date] = ["Professionnel", pro,  "en "+str(date)]
        month_pro_benef[date +"1"] = ["Bénéficiaire", benef, "en "+str(date)]
        month_pro_benef_percent[date] = ["Professionnel", round(pro/tous*100), "en "+str(date), str(round(pro/tous*100)) + "% ({})".format(str(pro).split(".")[0])]
        month_pro_benef_percent[date +"1"] = ["Bénéficiaire", round(benef/tous*100), "en "+str(date), str(round(benef/tous*100))+ "% ({})".format(str(benef).split(".")[0])]
    elif  semestre == "6":
      if ("2021-3" in date) or ("2021-4" in date) or ("2021-5" in date) or ("2021-6" in date) or ("2021-7" in date) or ("2021-8" in date): 
        benef = diffusion[diffusion["year-month"] == date]['Nb de bénéficiaires'].sum()
        pro = diffusion[diffusion["year-month"] == date]["Nb de pros"].sum()
        tous = benef + pro
        month_pro_benef[date] = ["Professionnel", pro,  "en "+str(date)]
        month_pro_benef[date +"1"] = ["Bénéficiaire", benef, "en "+str(date)]
        month_pro_benef_percent[date] = ["Professionnel", round(pro/tous*100), "en "+str(date), str(round(pro/tous*100)) + "% ({})".format(str(pro).split(".")[0])]
        month_pro_benef_percent[date +"1"] = ["Bénéficiaire", round(benef/tous*100), "en "+str(date), str(round(benef/tous*100))+ "% ({})".format(str(benef).split(".")[0])]
    else: 
      if ("2021-6" in date) or ("2021-7" in date) or ("2021-8" in date): 
        benef = diffusion[diffusion["year-month"] == date]['Nb de bénéficiaires'].sum()
        pro = diffusion[diffusion["year-month"] == date]["Nb de pros"].sum()
        tous = benef + pro
        month_pro_benef[date] = ["Professionnel", pro,  "en "+str(date)]
        month_pro_benef[date +"1"] = ["Bénéficiaire", benef, "en "+str(date)]
        month_pro_benef_percent[date] = ["Professionnel", round(pro/tous*100), "en "+str(date), str(round(pro/tous*100)) + "% ({})".format(str(pro).split(".")[0])]
        month_pro_benef_percent[date +"1"] = ["Bénéficiaire", round(benef/tous*100), "en "+str(date), str(round(benef/tous*100))+ "% ({})".format(str(benef).split(".")[0])]


  dff = pd.DataFrame(month_pro_benef).transpose().rename(columns={2: "Date de la diffusion", 1: "Nombre de personnes touchées"})
  #dff2 = pd.DataFrame(month_pro_benef_percent).transpose().rename(columns={2: "Date de la diffusion", 1: "% de personnes touchées"})
  dff = dff.replace("en 2021-3", "mars 2021")
  dff = dff.replace("en 2021-4", "avril 2021")
  dff = dff.replace("en 2021-5", "mai 2021")
  dff = dff.replace("en 2021-6", "juin 2021")
  dff = dff.replace("en 2021-7", "juillet 2021")
  dff = dff.replace("en 2021-8", "août 2021")

  fig = px.bar(dff, y="Nombre de personnes touchées",  x="Date de la diffusion", color=0,  barmode="group", text="Nombre de personnes touchées",     color_discrete_map={
          "Bénéficiaire": "#2896A0",
          "Professionnel":" #F5EBE1"}, height=700,
        hover_data=[0]).for_each_trace(lambda t: t.update(name=t.name.split("=")[1])
  )

  fig.update_layout( title_font_family="Arial")
  fig.update_layout(font=dict(
          family="Arial",
          size=18))

  fig.update_layout(template="plotly_white")
  fig.update_layout(legend=dict(
      orientation="h",
      yanchor="middle",
      y=-0.2,
      xanchor="center",
      x=0.5
  ))
  fig.update_layout(title_text=titre)

  #fig.update_traces(name="Fréquence d'utilisation hebdomadaires ou journalière", selector=dict(type='bar'))
  fig.show()

In [ ]:
def semestre_diffusion_dep(dep, answer = "maintenant"):
  return(semestre_diffusion(diffusion[diffusion["Territoire"] == dep], "Le nombre d'actions de diffusion en fonction du public visé dans le {}".format(dep),answer))

In [ ]:
month_pro_benef = dict()
month_pro_benef_percent = dict()
for date in np.unique(diffusion["year-month"]):
    benef = diffusion[diffusion["year-month"] == date]['Nb de bénéficiaires'].sum()
    pro = diffusion[diffusion["year-month"] == date]["Nb de pros"].sum()
    tous = benef + pro
    month_pro_benef[date] = [ pro,  benef]

    #month_pro_benef_percent[date] = ["Professionnel", round(pro/tous*100), "en "+str(date), str(round(pro/tous*100)) + "% ({})".format(str(pro).split(".")[0])]
    #month_pro_benef_percent[date +"1"] = ["Bénéficiaire", round(benef/tous*100), "en "+str(date), str(round(benef/tous*100))+ "% ({})".format(str(benef).split(".")[0])]


In [ ]:
df = pd.DataFrame(month_pro_benef).transpose()
df["date"] = df.index
df.sort_values(by="date")

# RECHERCHES MAP

In [ ]:
s_map = pd.DataFrame(s["Département"].value_counts()).rename(columns= {"Département":"Nombre de recherches"})
s_map["Code Postal"] = s_map.index

In [ ]:
s_map["Département"] = s_map["Code Postal"].apply(lambda x: str(x)[0:2])

In [ ]:
#s_map = s_map[s_map["Département"] == "33"]

In [ ]:
#s_map = s_map.replace(12139, 300)

In [ ]:
s_map.head(30)

In [ ]:
# codeFoireux(s_map)

In [ ]:
import folium
from folium.plugins import FloatImage

france = folium.Map(location = [46.232192999999995, 2.209666999999996], zoom_start = 6)
folium_del_legend(folium.Choropleth(data2, 
                  key_on = "feature.properties.code",
                  data = s_map.dropna(), 
                  columns = ["Code Postal", "Nombre de recherches"], 
                  fill_color='YlGnBu', 
                  fill_opacity=1,
                  line_opacity=0.4,
                  legend_name="Nbre de recherches effectuées sur Soliguide.fr",
                  smooth_factor=0,
                  Highlight= True,
                  line_color = "#e5e5e5",
                  name = "Structures répertoriées sur Soliguide.fr",
                  show=False,
                  overlay=True,
                  nan_fill_color = "White",
                  threshold_scale = [1,25,50,100,150,200, 500, 1000, 1000000],
                  )).add_to(france)


style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}
url_2 = ('https://raw.githubusercontent.com/Antoine-Carre/bdd_python/main/data_csv/Scale_nber_of_searches.png')
FloatImage(url_2, bottom=20, left=5).add_to(france)

france

In [ ]:
france.save("{}/search_map.html".format(path))

## Evolution des mises à jour Soliguide

In [ ]:
# Extraction des données de userDatas dans la base history
historyJSON['userDatas'] = historyJSON.userDatas.astype(str).apply(lambda x:eval(x))
df_history_bis =pd.json_normalize(historyJSON['userDatas'])
df_history_bis_2 = pd.concat([historyJSON, df_history_bis], axis=1)

In [ ]:
# Creation d'une table avec uniquement les colonnes od, created_at et status
df_history_2 = df_history_bis_2[['_id','created_at','status']]
df_history_2